In [30]:
import ee
import geemap
geemap.set_proxy(port = 56940)
# ee.Authenticate()
ee.Initialize()
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[40, -100], zoom=4)


In [41]:
def year_water_mapping(threshold,s=2014,e = 2022,):
    roi = ee.Geometry.Polygon( [ [   [180, 90],
                                    [180, -90],
                                    [-180 , -90] ,
                                    [-180, 90],
                            ]],None,False,)
    Map1 = geemap.Map(center=[40, -100], zoom=4)
    for i in range(s,e):
        print(i)
        start_ = str(i)+'-01-01'
        end_ = str(i+1)+'-01-01'
        start = ee.Date(start_)
        finish = ee.Date(end_)
        filteredCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterBounds(roi) \
    .filterDate(start, finish) \
    .filter(ee.Filter.lt("CLOUD_COVER",20))\
 .sort('CLOUD_COVER', True)
   

        mosaic = filteredCollection.mosaic()

        #Map = geemap.Map(center=[30, 90], zoom=5)
        vizParams = {'bands': ['B5', 'B4', 'B3'],
                     'min': 5000, 'max': 15000, 'gamma': 1.3}
        first = mosaic
#wi = first.expression('(3*g-1*b+2*r-5*nir-swir2)/(3*g+1*b+2*r+5*nir+swir2)',{
        wi = first.expression('(1*g-0.35*b-0.35*r-1.1*nir)**2/(1*g+0.35*b+0.35*r+1.1*nir)**2',{
            'b':first.select('B2'),
          'g':first.select('B3'),
          'r':first.select('B4'),
          'nir':first.select('B5'),
          'swir1':first.select('B6'),
          'swir2':first.select('B7')
        })
        vizParams2 = {'min': -1, 'max': 1, 'gamma': 1.3}
        #vizParams2 = {}
        winame = 'wi ' +str(i)
        Map1.addLayer(wi, vizParams2, winame,0)
        L8name = 'Landsat 8 image ' +str(i)
        Map1.addLayer(mosaic, vizParams,L8name,0)
        palette = ['42C8F0','29D69F','3228D6','DC23EB','DC23FA','3758A9']
        wi_mask = wi.updateMask(wi.lt(threshold))  #lt
        maskname = 'wi_mask ' + str(i)
        import random
        m =random.randint(0,5)
        print(palette[m])
        Map1.addLayer(wi_mask, {'min': 0, 'max': 1,'palette':palette[m]}, maskname,1)
#     Map1.addLayer(roi,{}, 'roi')
    return Map1

In [42]:
Map1 = year_water_mapping(0.075,2017,2018)

# roi = ee.Geometry.Polygon( [ [   [180, 90],
#                                     [180, -90],
#                                     [-180 , -90] ,
#                                     [-180, 90],
#                             ]],None,False,)
# Map1.addLayer(roi,{}, 'roi')

2017
29D69F


In [43]:
Map1

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…